In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
torch.backends.cudnn.benchmark = True

In [2]:
import deepdish as dd
from sequence_annotation.genome_handler.seq_container import AnnSeqContainer
from sequence_annotation.data_handler.fasta import read_fasta
from sequence_annotation.genome_handler.utils import select_seq
from sequence_annotation.genome_handler.ann_genome_processor import get_mixed_types_genome,simplify_genome
from sequence_annotation.visual.visual import visual_ann_genome,visual_ann_seq
from sequence_annotation.pytorch.SA_facade import SeqAnnFacade
from sequence_annotation.pytorch.loss import SeqAnnLoss
from sequence_annotation.genome_handler import ann_seq_processor
from sequence_annotation.pytorch.model import SeqAnnModel,seq_ann_inference
from sequence_annotation.data_handler.fasta import write_fasta

Using TensorFlow backend.


In [3]:
h5=dd.io.load('../io/Arabidopsis_thaliana/data/2019_05_11/result/selected_region.h5')
fasta = read_fasta('../io/Arabidopsis_thaliana/data/2019_05_11/result/selected_region.fasta')
ann_seqs = AnnSeqContainer().from_dict(h5)

In [4]:
ann_seqs = get_mixed_types_genome(ann_seqs)
simplify_map={'exon':['cds','utr_5','utr_3','mix'],'intron':['intron'],'other':['other']}
ann_seqs = simplify_genome(ann_seqs,simplify_map)
selected_fasta,selected_seqs,o_f,o_s = select_seq(fasta,ann_seqs,max_len=2000)

/home/sequence_annotation/sequence_annotation/genome_handler/ann_seq_processor.py:238: RuntimeWarning: invalid value encountered in true_divide
  vecs.append(np.nan_to_num(temp/temp))


In [5]:
facade = SeqAnnFacade()
facade.assign_data_by_random(selected_fasta,selected_seqs,ratio=[0.7,0.3,0])

278 {'exon': 182591.0, 'intron': 20899.0, 'other': 278000.0}
118 {'exon': 81031.0, 'intron': 9514.0, 'other': 118000.0}


In [6]:
len(facade.train_seqs)

278

In [7]:
facade.set_path('../io/record/arabidopsis_2019_05_15/debug')
facade.set_optimizer(optimizer_settings={'lr':1e-3})
facade.simplify_map = {'gene':['exon','intron'],'other':['other']}
facade.add_seq_fig(o_f,o_s,inference=None)

/home/sequence_annotation/sequence_annotation/genome_handler/ann_seq_processor.py:191: UserWarning: 

!!!
	DNA sequence will be rearranged from 5' to 3'.
	The plus strand sequence will stay the same, but the minus strand sequence will be flipped!
!!!

  warnings.warn(warn)


In [8]:
write_fasta('../io/record/arabidopsis_2019_05_22/trial_01/train.fasta',facade.train_seqs)
write_fasta('../io/record/arabidopsis_2019_05_22/trial_01/val.fasta',facade.val_seqs)

In [9]:
space={
    'cnns_setting':{
        "ln_mode":"after_activation",
        'num_layers':4,
        'with_pwm':False
        'out_channels':[16,16,16,16],
        'kernel_sizes':[60,60,60,60]
    },
    'rnns_setting':{
        'num_layers':4,
        'hidden_size':32,
        'batch_first':True,
        'bidirectional':True
    },
    'pwm_before_rnns':False,
    'rnns_type':'GRU',
    'last_kernel_size':1
}

In [10]:
model = SeqAnnModel(input_size=4,out_channels=3,use_sigmoid=True,**space).cuda()

In [11]:
train_record = facade.train(model,batch_size=128,epoch_num=20)

278 {'exon': 182591.0, 'intron': 20899.0, 'other': 278000.0}
118 {'exon': 81031.0, 'intron': 9514.0, 'other': 118000.0}


/home/sequence_annotation/sequence_annotation/genome_handler/ann_genome_processor.py:75: UserWarning: 

!!!
	DNA sequence will be rearranged from 5' to 3'.
	The plus strand sequence will stay the same, but the minus strand sequence will be flipped!
!!!

  warnings.warn(warn)


Creating folder ../io/record/arabidopsis_2019_05_15/debug...
Processing worker...
Executing...
Start working(2019-05-18 17:41:30)
1 21.467 {'macro_F1': 0.387, 'F1_intron': 0.0, 'F1_other': 0.003, 'accuracy': 0.381, 'F1_exon': 0.551, 'val_accuracy': 0.546, 'loss': 3.196, 'val_macro_F1': 0.32, 'val_F1_intron': 0.0, 'val_F1_exon': 0.006, 'val_loss': 1.463, 'val_F1_other': 0.706}
2 23.805 {'macro_F1': 0.372, 'F1_intron': 0.0, 'F1_other': 0.188, 'accuracy': 0.425, 'F1_exon': 0.559, 'val_accuracy': 0.453, 'loss': 1.31, 'val_macro_F1': 0.376, 'val_F1_intron': 0.0, 'val_F1_exon': 0.586, 'val_loss': 1.033, 'val_F1_other': 0.21}
3 22.755 {'macro_F1': 0.354, 'F1_intron': 0.156, 'F1_other': 0.716, 'accuracy': 0.557, 'F1_exon': 0.002, 'val_accuracy': 0.5, 'loss': 0.95, 'val_macro_F1': 0.437, 'val_F1_intron': 0.172, 'val_F1_exon': 0.125, 'val_loss': 0.991, 'val_F1_other': 0.672}
4 22.538 {'macro_F1': 0.421, 'F1_intron': 0.0, 'F1_other': 0.643, 'accuracy': 0.61, 'F1_exon': 0.591, 'val_accuracy': 0.61